In [75]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
%matplotlib inline
import tensorflow as tf
from scipy import stats
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
import random
import pickle

import seaborn as sns; sns.set()

# Data set cleaning
1) Cropping the faces from the data  
2) Regenerate the data with faces only  

In [27]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

In [28]:
def get_cropped_image_if_2_eyes(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        if len(eyes) >= 2:
            return roi_color

In [29]:
path_to_data = "./data/"
path_to_cr_data = "./data/cropped/"

In [30]:
import os
img_dirs = []
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        img_dirs.append(entry.path)

In [31]:
img_dirs

['./data/train/cropped', './data/train/without_mask', './data/train/with_mask']

In [32]:
import shutil
if os.path.exists(path_to_cr_data):
     shutil.rmtree(path_to_cr_data)
os.mkdir(path_to_cr_data)

In [ ]:
cropped_image_dirs = []
file_names_dict = {}

for img_dir in img_dirs:
    count = 1
    name = img_dir.split('/')[-1]
    print(name)
    
    file_names_dict[name] = []
    
    for entry in os.scandir(img_dir):
        roi_color = get_cropped_image_if_2_eyes(entry.path)
        if roi_color is not None:
            cropped_folder = path_to_cr_data + name
            if not os.path.exists(cropped_folder):
                os.makedirs(cropped_folder)
                cropped_image_dirs.append(cropped_folder)
                print("Generating cropped images in folder: ",cropped_folder)
                
            cropped_file_name = name + str(count) + ".png"
            cropped_file_path = cropped_folder + "/" + cropped_file_name 
            
            cv2.imwrite(cropped_file_path, roi_color)
            file_names_dict[name].append(cropped_file_path)
            count += 1    

cropped
without_mask
Generating cropped images in folder:  ./data/train/cropped/without_mask
with_mask
Generating cropped images in folder:  ./data/train/cropped/with_mask


# Image processing and labelling
1) Load the data

2) Save the data   

3) Image processing

4) Label the data

5) Split train and test data

 

In [76]:
IMG_SIZE=100

In [3]:


DATADIR = "data"

CATEGORIES = ["with_mask", "without_mask"]
data = []
for category in CATEGORIES:  

        path = os.path.join(DATADIR,category)  
        class_num = CATEGORIES.index(category)  

        for img in tqdm(os.listdir(path)):  
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  
                data.append([new_array, class_num])  
            except Exception as e:  
                pass



100%|██████████| 33814/33814 [06:42<00:00, 84.03it/s] 


In [4]:
filename = 'data1'
outfile = open(filename,'wb')
pickle.dump(data,outfile)
outfile.close()


In [77]:
infile = open('data1','rb')
data = pickle.load(infile)
infile.close()


In [78]:
random.shuffle(data)

In [79]:
X = []
Y = []

for features,label in data:
    X.append(features)
    Y.append(label)


X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = np.array(Y)
X = X / 255.0

print(X.shape)
print(Y.shape)


(63136, 100, 100, 1)
(63136,)


In [80]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size = 0.20)

# Model training and testing  

1) Trane the convolutional neural network 

2) Test the convolutional neural network 

In [82]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
from keras import optimizers

In [92]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(IMG_SIZE, IMG_SIZE)),
    tf.keras.layers.Dense(124, activation='tanh'),
    tf.keras.layers.Dense(124, activation='tanh'),
    tf.keras.layers.Dense(1 ,activation='sigmoid')
])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 124)               1240124   
_________________________________________________________________
dense_16 (Dense)             (None, 124)               15500     
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 125       
Total params: 1,255,749
Trainable params: 1,255,749
Non-trainable params: 0
_________________________________________________________________


In [93]:
model.compile(optimizer=optimizers.RMSprop(lr=0.00001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [94]:
model.fit(xtrain, ytrain, epochs=100)

Epoch 1/100
1579/1579 [==============================] - 24s 15ms/step - loss: 0.2398 - accuracy: 0.9303
Epoch 2/100
1579/1579 [==============================] - 24s 15ms/step - loss: 0.1086 - accuracy: 0.96711s - loss: 0
Epoch 3/100
1579/1579 [==============================] - 23s 15ms/step - loss: 0.0863 - accuracy: 0.9733
Epoch 4/100
1579/1579 [==============================] - 23s 14ms/step - loss: 0.0733 - accuracy: 0.9764
Epoch 5/100
1579/1579 [==============================] - 22s 14ms/step - loss: 0.0656 - accuracy: 0.9796
Epoch 6/100
1579/1579 [==============================] - 23s 15ms/step - loss: 0.0604 - accuracy: 0.9812
Epoch 7/100
1579/1579 [==============================] - 23s 15ms/step - loss: 0.0564 - accuracy: 0.9830
Epoch 8/100
1579/1579 [==============================] - 24s 15ms/step - loss: 0.0535 - accuracy: 0.9838
Epoch 9/100
1579/1579 [==============================] - 23s 15ms/step - loss: 0.0514 - accuracy: 0.9846
Epoch 10/100
1579/1579 [===================

1579/1579 [==============================] - 24s 15ms/step - loss: 0.0144 - accuracy: 0.9961
Epoch 78/100
1579/1579 [==============================] - 23s 14ms/step - loss: 0.0141 - accuracy: 0.9960
Epoch 79/100
1579/1579 [==============================] - 23s 14ms/step - loss: 0.0138 - accuracy: 0.9965
Epoch 80/100
1579/1579 [==============================] - 22s 14ms/step - loss: 0.0132 - accuracy: 0.9966
Epoch 81/100
1579/1579 [==============================] - 23s 14ms/step - loss: 0.0139 - accuracy: 0.9962
Epoch 82/100
1579/1579 [==============================] - 22s 14ms/step - loss: 0.0135 - accuracy: 0.9967
Epoch 83/100
1579/1579 [==============================] - 23s 14ms/step - loss: 0.0135 - accuracy: 0.9961
Epoch 84/100
1579/1579 [==============================] - 22s 14ms/step - loss: 0.0129 - accuracy: 0.9966
Epoch 85/100
1579/1579 [==============================] - 22s 14ms/step - loss: 0.0127 - accuracy: 0.9968
Epoch 86/100
1579/1579 [==============================] - 2

In [95]:
test_loss, test_acc = model.evaluate(xtest,  ytest, verbose=2)

print('\nTest accuracy:', test_acc)

395/395 - 2s - loss: 0.0234 - accuracy: 0.9946

Test accuracy: 0.9946151375770569


In [96]:
model.save("mask model 9", save_format="h5")

# Life cam and prediction

1) Recognising face using life camera

2) Predict if the face wearing a mask or not 

3) Show the result 

In [53]:
from keras.models import load_model
import cv2
import numpy as np

In [115]:
model = load_model('mask model 9')


In [120]:


face_clsfr = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source = cv2.VideoCapture(0)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,3)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(IMG_SIZE,IMG_SIZE))
        normalized=resized/255
        reshaped=np.reshape(normalized,(-1,IMG_SIZE,IMG_SIZE,1))
        ar = np.expand_dims(reshaped, axis = 0)
        result=model.predict(ar)

        
       
        
        if  result <=0.999:
            label=0
            
        else:
            label=1
        
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

# Life cam prediction and gate controlling 

1) Recognising face using life camera

2) Predict if the face wearing a mask or not 

3) Show the result

4) Open and close the gait depending on the result

In [23]:
from pyfirmata import Arduino, SERVO,util

In [34]:
pin=10
board=Arduino('COM3')

board.digital[pin].mode=SERVO


    
def gatard(x):
    if x==0:
        board.digital[10].write(90)
       
    elif x==1:
        board.digital[10].write(0)
  
        
        

In [35]:
model = load_model('mask model 9')

In [105]:
face_clsfr = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source = cv2.VideoCapture(0)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  
    
    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(IMG_SIZE,IMG_SIZE))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(-1,IMG_SIZE,IMG_SIZE,1))
        ar = np.expand_dims(reshaped, axis = 0)
        result=model.predict(ar)

        
     
        
        if  result <= 0.98:
            label=0
            
        else:
            label=1
        
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        gatard(label)
      
        
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-u4kjpz2z\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
